In [1]:
#!pip install pvbm --upgrade --quiet
import os
import pandas as pd
import matplotlib.pyplot as plt
import PVBM
import numpy as np
from PIL import Image

from skimage.morphology import skeletonize

In [2]:
IMAGES_PATH = '/oriondata/AIMLab/renee.zac/Datasets/mbrset/mbrset-a-mobile-brazilian-retinal-dataset-1.0/images/'


In [3]:
SEGMENTATIONS_PATH = './mbrset_lunout'

In [4]:
from PVBM.GeometryAnalysis import GeometricalVBMs #Import the geometry analysis module
geometricalVBMs = GeometricalVBMs()

In [5]:
from PVBM.DiscSegmenter import DiscSegmenter
segmenter = DiscSegmenter()

Model path: /home/renee.zac/miniforge3/envs/expl3nv.12/lib/python3.12/site-packages/PVBM/lunetv2_odc.onnx
Model already exists, skipping download.


In [7]:
from tqdm import tqdm

In [8]:
vbm_df = pd.DataFrame(columns = [f"{vessel}_{metric}" for vessel in ["Artery", "Vein"] for metric in [
	"Area", "Tortuosity_Index", "Median_Tortuosity", "Overall_Length", "Median_Branching_Angles", "N_Start_Points","N_End_Points","N_Intersection_Points"
]]+ ['Index'])


image_files = os.listdir(IMAGES_PATH)
image_files = [f for f in image_files if f.endswith('.jpg')]
for img_file in tqdm(image_files):
    # Load the image and segmentation
     # take the name without extension

    img = Image.open(os.path.join(IMAGES_PATH, img_file))

    sample = os.path.splitext(img_file)[0]
    seg_file = os.path.join(SEGMENTATIONS_PATH, sample + '.npy')
    seg = np.load(seg_file)

    # resize the image to match the segmentation shape
    img = img.resize((seg.shape[0], seg.shape[1]))

    artery_seg = (seg[:,:,0] > 0).astype(np.uint8)
    skeleton_artery = skeletonize(artery_seg)
    skeleton_artery = skeleton_artery.astype(np.uint8)

    vein_seg = (seg[:,:,2] > 0).astype(np.uint8)
    skeleton_vein = skeletonize(vein_seg)
    skeleton_vein = skeleton_vein.astype(np.uint8)

    # Extract the optic disc
    #segmenter = DiscSegmenter()
    optic_disc = segmenter.segment(image_path=os.path.join(IMAGES_PATH, img_file))
    optic_disc = optic_disc.resize((seg.shape[0], seg.shape[1]))
    center, radius, roi, zones_ABC = segmenter.post_processing(segmentation=optic_disc, max_roi_size=600) 

    #geometricalVBMs = GeometricalVBMs()
    artery_segmentation_roi, artery_skeleton_roi = geometricalVBMs.apply_roi(
    segmentation=artery_seg,
    skeleton=skeleton_artery,
    zones_ABC=zones_ABC,
    roi=roi
)
    
    artery_vbms, artery_visual = geometricalVBMs.compute_geomVBMs(
    blood_vessel=artery_segmentation_roi,
    skeleton=artery_skeleton_roi,
    xc=center[0],
    yc=center[1],
    radius=radius
)
    artery_area, artery_TI, artery_medTor, artery_ovlen, artery_medianba, artery_startp, artery_endp, artery_interp = artery_vbms
    
    vein_segmentation_roi, vein_skeleton_roi = geometricalVBMs.apply_roi(
    segmentation=vein_seg,
    skeleton=skeleton_vein,
    zones_ABC=zones_ABC,
    roi=roi
)
    
    vein_vbms, vein_visual = geometricalVBMs.compute_geomVBMs(
    blood_vessel=vein_segmentation_roi,
    skeleton=vein_skeleton_roi,
    xc=center[0],
    yc=center[1],
    radius=radius
)
    vein_area, vein_TI, vein_medTor, vein_ovlen, vein_medianba, vein_startp, vein_endp, vein_interp = vein_vbms

    vbm_df = pd.concat([vbm_df,pd.DataFrame([{'Index':str(sample),'Artery_Area':artery_area, 'Artery_Tortuosity_Index':artery_TI, 'Artery_Median_Tortuosity':artery_medTor,
        'Artery_Overall_Length':artery_ovlen, 'Artery_Median_Branching_Angles':artery_medianba,
        'Artery_N_Start_Points':artery_startp, 'Artery_N_End_Points':artery_endp,
        'Artery_N_Intersection_Points':artery_interp, 'Vein_Area':vein_area, 'Vein_Tortuosity_Index':vein_TI,
        'Vein_Median_Tortuosity':vein_medTor, 'Vein_Overall_Length':vein_ovlen,
        'Vein_Median_Branching_Angles':vein_medianba, 'Vein_N_Start_Points':vein_startp,
        'Vein_N_End_Points':vein_endp, 'Vein_N_Intersection_Points':vein_interp}])],ignore_index=True)



  0%|          | 0/5164 [00:00<?, ?it/s]

/tmp/ipykernel_2166696/212916863.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vbm_df = pd.concat([vbm_df,pd.DataFrame([{'Index':str(sample),'Artery_Area':artery_area, 'Artery_Tortuosity_Index':artery_TI, 'Artery_Median_Tortuosity':artery_medTor,
  0%|          | 12/5164 [00:59<7:02:16,  4.92s/it]/home/renee.zac/miniforge3/envs/expl3nv.12/lib/python3.12/site-packages/PVBM/GeometryAnalysis.py:449: RuntimeWarning: invalid value encountered in scalar divide
  TI = arc_list.sum() / chord_list.sum()
/home/renee.zac/miniforge3/envs/expl3nv.12/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/renee.zac/miniforge3/envs/expl3nv.12/lib/pyt

In [9]:
vbm_df.head()

,Artery_Area,Artery_Tortuosity_Index,Artery_Median_Tortuosity,Artery_Overall_Length,Artery_Median_Branching_Angles,Artery_N_Start_Points,Artery_N_End_Points,Artery_N_Intersection_Points,Vein_Area,Vein_Tortuosity_Index,Vein_Median_Tortuosity,Vein_Overall_Length,Vein_Median_Branching_Angles,Vein_N_Start_Points,Vein_N_End_Points,Vein_N_Intersection_Points,Index
0,20578.0,1.101783,1.075637,1555.893578,70.575740,4,5.0,1.0,49007.0,1.092587,1.084222,4196.864499,77.275644,4,13.0,12.0,47.4
1,49634.0,1.103772,1.087451,5520.839246,91.702621,9,17.0,9.0,57754.0,1.093053,1.081833,6274.276980,68.813001,11,25.0,14.0,378.3
2,39278.0,1.077895,1.074380,5061.986433,69.167245,9,14.0,5.0,47080.0,1.093832,1.079156,5861.105407,66.551865,8,19.0,13.0,207.1
3,34729.0,1.091477,1.084032,5386.770270,92.364036,7,13.0,5.0,38741.0,1.099560,1.081481,5342.461613,79.777831,8,16.0,11.0,1075.2
4,25567.0,1.064010,1.069275,2744.438959,62.311646,6,9.0,4.0,23127.0,1.079073,1.081001,2076.973880,74.009028,4,5.0,1.0,1023.3


In [10]:
len(vbm_df)

5164

In [11]:
vbm_df.to_csv('mbrset_vbm_df.csv', index=False)

,Artery_Area,Artery_Tortuosity_Index,Artery_Median_Tortuosity,Artery_Overall_Length,Artery_Median_Branching_Angles,Artery_N_Start_Points,Artery_N_End_Points,Artery_N_Intersection_Points,Vein_Area,Vein_Tortuosity_Index,Vein_Median_Tortuosity,Vein_Overall_Length,Vein_Median_Branching_Angles,Vein_N_Start_Points,Vein_N_End_Points,Vein_N_Intersection_Points,Index
0,20578.0,1.101783,1.075637,1555.893578,70.575740,4,5.0,1.0,49007.0,1.092587,1.084222,4196.864499,77.275644,4,13.0,12.0,47.4
0,49634.0,1.103772,1.087451,5520.839246,91.702621,9,17.0,9.0,57754.0,1.093053,1.081833,6274.276980,68.813001,11,25.0,14.0,378.3
0,39278.0,1.077895,1.074380,5061.986433,69.167245,9,14.0,5.0,47080.0,1.093832,1.079156,5861.105407,66.551865,8,19.0,13.0,207.1
0,34729.0,1.091477,1.084032,5386.770270,92.364036,7,13.0,5.0,38741.0,1.099560,1.081481,5342.461613,79.777831,8,16.0,11.0,1075.2
0,25567.0,1.064010,1.069275,2744.438959,62.311646,6,9.0,4.0,23127.0,1.079073,1.081001,2076.973880,74.009028,4,5.0,1.0,1023.3
